<center>
    <h1><font size="16">ESCUELA DE ECONOMÍA Y ADMINISTRACIÓN</font></h1>
    <h1><font size="12">ECONOMETRÍA DE SERIES DE TIEMPO</font></h1>
</center>

<h2>🧪 Taller práctico: Métodos de Suavizamiento y Regresión</h2>

<h3>🎯 Objetivo</h3>

Aplicar modelos de suavizamiento (Holt-Winters) y regresión lineal sobre una serie temporal real, evaluando su ajuste, pronóstico y validez.

**📦 Dataset sugerido**

- `AirPassengers` (disponible por defecto)
- O una serie personalizada que contenga tendencia y estacionalidad clara (ventas mensuales, consumo eléctrico, visitantes, etc.).

---

<h3>🧭 Actividades</h3>

🔹 **1. Exploración inicial**

1.1. Carga la serie y conviértela a un objeto `tsibble`  
1.2. Visualiza la serie original y describe brevemente su comportamiento


In [ ]:
# librerías necesarias
library(forecast)
library(ggplot2)
library(fpp3)
library(yardstick)
library(dplyr)

In [ ]:
ap <- as_tsibble(AirPassengers)

autoplot(ap) +
  ggtitle("Serie original: pasajeros mensuales") +
  theme_minimal()


🔹 **2. Suavizamiento con Holt-Winters**

2.1. Ajusta dos modelos Holt-Winters:

- Uno con componente **aditiva**
- Otro con componente **multiplicativa**

2.2. Compara los pronósticos a 12 pasos vista


In [ ]:
modelo_hw_add <- hw(AirPassengers, seasonal = "additive", h = 12)
modelo_hw_mult <- hw(AirPassengers, seasonal = "multiplicative", h = 12)

autoplot(modelo_hw_add) + ggtitle("Holt-Winters aditivo") + theme_minimal()
autoplot(modelo_hw_mult) + ggtitle("Holt-Winters multiplicativo") + theme_minimal()


2.3. ¿Cuál se ajusta mejor a la dinámica observada?

---

🔹 **3. Regresión lineal simple y múltiple**

3.1. Crea una variable de tendencia:


In [ ]:
ap <- ap %>%
  mutate(t = row_number())


3.2. Ajusta un modelo de regresión lineal simple:  
$$
Y_t = \beta_0 + \beta_1 t + \varepsilon_t
$$

3.3. Ajusta un modelo múltiple con estacionalidad:

In [ ]:
modelo_rlm <- ap %>%
  model(RLM = TSLM(value ~ t + season()))

report(modelo_rlm)

3.4. Genera el `new_data()` y calcula el pronóstico

In [ ]:
new_data <- ap %>%
  new_data(n = 12) %>%
  mutate(t = max(ap$t) + 1:12)

forecast(modelo_rlm, new_data = new_data) %>%
  autoplot(ap) +
  theme_minimal()


🔹 **4. Análisis de residuos**

4.1. Visualiza residuos del modelo RLM:


In [ ]:
modelo_rlm %>% gg_tsresiduals()

4.2. Comenta si los residuos parecen ruido blanco, o si hay evidencia de autocorrelación o patrón estacional no capturado.

---

🔹 **5. Comparación final**

5.1. Calcula las métricas de precisión (`accuracy()`) para Holt-Winters aditivo, multiplicativo y el modelo RLM (si es posible).


In [ ]:
# library(forecast)

forecast::accuracy(modelo_hw_add) %>% t() %>% print()
forecast::accuracy(modelo_hw_mult) %>% t() %>% print()

In [ ]:
# library(yardstick)
# library(dplyr)

# Obtener valores ajustados
ajuste_rlm <- augment(modelo_rlm)

# Calcular métricas comparando truth y estimate
ajuste_rlm %>%
  metrics(truth = value, estimate = .fitted) %>%
  print()


In [ ]:
# Obtener accuracy() para modelos Holt-Winters
acc_hw_add <- forecast::accuracy(modelo_hw_add) %>%
  as_tibble(rownames = "Conjunto") %>%
  filter(Conjunto == "Training set") %>%
  transmute(
    Modelo = "Holt-Winters Aditivo",
    RMSE = RMSE,
    MAE = MAE,
    MAPE = MAPE,
    ACF1 = ACF1,
    R2 = NA_real_
  )

acc_hw_mult <- forecast::accuracy(modelo_hw_mult) %>%
  as_tibble(rownames = "Conjunto") %>%
  filter(Conjunto == "Training set") %>%
  transmute(
    Modelo = "Holt-Winters Multiplicativo",
    RMSE = RMSE,
    MAE = MAE,
    MAPE = MAPE,
    ACF1 = ACF1,
    R2 = NA_real_
  )

# Obtener métricas del modelo de regresión (ajuste en muestra)
acc_rlm <- ajuste_rlm %>%
  metrics(truth = value, estimate = .fitted) %>%
  select(.metric, .estimate) %>%
  pivot_wider(names_from = .metric, values_from = .estimate) %>%
  transmute(
    Modelo = "Regresión Lineal (TSLM)",
    RMSE = rmse,
    MAE = mae,
    MAPE = NA_real_,
    ACF1 = NA_real_,
    R2 = rsq
  )

# Unir todo en una tabla comparativa
tabla_modelos <- bind_rows(acc_hw_add, acc_hw_mult, acc_rlm)

# Mostrar
print(tabla_modelos)


5.2. ¿Cuál modelo resulta más adecuado para esta serie?

---

<h3>🧠 Preguntas de reflexión</h3>

- ¿Cómo influye la elección aditiva vs multiplicativa en la calidad del pronóstico?
- ¿Qué ventajas aporta un modelo de regresión frente a uno puramente estructural como Holt-Winters?
- ¿Qué harías si los residuos presentan estacionalidad no explicada?
